<a href="https://colab.research.google.com/github/pandian-raja/EVA4_Session14-15A-/blob/master/DenseFinalOutput.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ialhashim/DenseDepth.git

Cloning into 'DenseDepth'...
remote: Enumerating objects: 235, done.
remote: Total 235 (delta 0), reused 0 (delta 0), pack-reused 235
Receiving objects: 100% (235/235), 11.80 MiB | 34.91 MiB/s, done.
Resolving deltas: 100% (115/115), done.


In [0]:
from PIL import Image
import numpy as np
from timeit import default_timer as timer
import datetime
import shutil
import zipfile
from zipfile import ZipFile 
import numpy as np
from PIL import Image




In [2]:
!wget https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5 -O ./DenseDepth/nyu.h5

--2020-06-03 09:47:52--  https://s3-eu-west-1.amazonaws.com/densedepth/nyu.h5
Resolving s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)... 52.218.80.178
Connecting to s3-eu-west-1.amazonaws.com (s3-eu-west-1.amazonaws.com)|52.218.80.178|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172897376 (165M) [application/h5]
Saving to: ‘./DenseDepth/nyu.h5’

./DenseDepth/nyu.h5 100%[===================>] 164.89M  24.4MB/s    in 7.4s    

2020-06-03 09:47:59 (22.2 MB/s) - ‘./DenseDepth/nyu.h5’ saved [172897376/172897376]



In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
zip = ZipFile('/content/drive/My Drive/copy_depth_fg_bg.zip')
zip.extractall('/tmp')

In [4]:
import os
import glob
import argparse
import matplotlib

# Keras / TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '5'
from keras.models import load_model
from DenseDepth.layers import BilinearUpSampling2D
from DenseDepth.utils import predict, load_images, display_images
from matplotlib import pyplot as plt

# Argument Parser
parser = argparse.ArgumentParser(description='High Quality Monocular Depth Estimation via Transfer Learning')
parser.add_argument('--model', default='DenseDepth/nyu.h5', type=str, help='Trained Keras model file.')
parser.add_argument('--input', default='DenseDepth/examples/*.png', type=str, help='Input filename or folder.')
# args = parser.parse_args()

args = parser.parse_args(args=[])

# Custom object needed for inference and training
custom_objects = {'BilinearUpSampling2D': BilinearUpSampling2D, 'depth_loss_function': None}

print('Loading model...')

# Load model into GPU / CPU
model = load_model(args.model, custom_objects=custom_objects, compile=False)
def to_multichannels(i):
    if i.shape[2] == 3: return i
    i = i[:,:,0]
    return np.stack((i,i,i), axis=2)

Using TensorFlow backend.


Loading model...


In [13]:
folder_path = '/tmp/fg_bg/'
globalstart = timer()
fincount =1
out_zip =  ZipFile('depth_fin_400k.zip', mode='a')
for imageBatch in range(100):
    print('Batch No : '+str(imageBatch))
    loaded_images = []
    rangestart = imageBatch*4000
    rangestart +=1
    start = timer()
    for imgc in range(rangestart,rangestart+4000):
        x = np.clip(np.asarray(Image.open( folder_path+str(imgc)+'.jpeg' ), dtype=float) / 255, 0, 1)
        loaded_images.append(x)
    loaded_images = np.stack(loaded_images, axis=0)  
    print('Loading Image time : '+str(timer() - start))
    start = timer()
    outputs = predict(model, loaded_images,batch_size=256)
    print('prediction time : '+str(timer() - start))
    start = timer()
    for x in range(outputs.shape[0]):
        montage = to_multichannels(outputs[x])
        im = Image.fromarray(np.uint8(montage*255))
        im.save('depth.jpeg', format="jpeg", optimize=True, quality=95)
        out_zip.write('depth.jpeg', f'depth/'+str(fincount)+'.jpeg')
        fincount +=1
    print('Image zip/saving time : '+str(timer() - start))
out_zip.close()
print('Total time : '+str(timer() - globalstart))

Batch No : 0
Loading Image time : 36.132210203
prediction time : 53.06883398500008
Image zip/saving time : 3.8980215230000113
Batch No : 1
Loading Image time : 26.69180743000004
prediction time : 27.659211486999993
Image zip/saving time : 4.00977796899997
Batch No : 2
Loading Image time : 23.769023500000003
prediction time : 27.65870698399999
Image zip/saving time : 3.8811908540000104
Batch No : 3
Loading Image time : 22.204700523000042
prediction time : 27.678284183000073
Image zip/saving time : 4.531788448000043
Batch No : 4
Loading Image time : 21.703550304000032
prediction time : 27.701897020999922
Image zip/saving time : 3.872721535999972
Batch No : 5
Loading Image time : 22.042337733999943
prediction time : 27.69335278799997
Image zip/saving time : 3.978099752999924
Batch No : 6
Loading Image time : 21.333835162000014
prediction time : 27.71501080099995
Image zip/saving time : 3.694285370999978
Batch No : 7
Loading Image time : 21.322349265999947
prediction time : 27.736503998999

In [16]:
shutil.move('depth_fin_400k.zip','/content/drive/My Drive/depth_fin_400k.zip')

'/content/drive/My Drive/depth_fin_400k.zip'

In [80]:
traindata = torchvision.datasets.ImageFolder( 'fg_bg1/fg_bg/', transforms.ToTensor())
image_means = torch.stack([t.mean(1).mean(1) for t, c in traindata])

print(image_means.mean(0))
print(image_means.std(0))

tensor([0.5373, 0.4887, 0.4405])
tensor([0.1111, 0.1161, 0.1190])


In [81]:
maskdata = torchvision.datasets.ImageFolder( 'mask_fg_bg1/mask_fg_bg/', transforms.ToTensor())
mask = torch.stack([t.mean(1).mean(1) for t, c in maskdata])

print(mask.mean(0))
print(mask.std(0))

tensor([0.0352, 0.0353, 0.0353])
tensor([0.0178, 0.0179, 0.0180])


In [82]:
depthdata = torchvision.datasets.ImageFolder( 'depth1/depth/', transforms.ToTensor())
depth = torch.stack([t.mean(1).mean(1) for t, c in depthdata])
print(depth.mean(0))
print(depth.std(0))

tensor([0.3627, 0.3627, 0.3627])
tensor([0.0827, 0.0827, 0.0827])


In [93]:

bgdata = torchvision.datasets.ImageFolder( 'bg1/', transforms.ToTensor())
bg = torch.stack([t.mean(1).mean(1) for t, c in depthdata])
bg.mean(0)


tensor([0.3627, 0.3627, 0.3627])

In [0]:
start = timer()
inputs = load_images( glob.glob(args.input) )
print('\nLoaded ({0}) images of size {1}.'.format(inputs.shape[0], inputs.shape[1:]))
duration = timer() - start
# Compute results


In [0]:

for x in range(outputs.shape[0]):
    montage = to_multichannels(outputs[x])
    im = Image.fromarray(np.uint8(montage*255))
    im.save('depth/'+str(fincount)+'.jpeg')
    fincount +=1


In [0]:
outputs = predict(model, inputs,batch_size=100)
for x in range(outputs.shape[0]):
    montage = to_multichannels(outputs[x])
    im = Image.fromarray(np.uint8(montage*255))
    im.save('depth/'+str(fincount)+'.jpeg')
    fincount +=1


In [0]:
import os
for x in range(400000):
    myfile = 'mask_fg_bg/'+str(x)+'.jpeg'
    if os.path.isfile(myfile):
        os.remove(myfile)

In [0]:
outputs.shape

(4000, 112, 112, 1)

In [0]:
for x in range(4):
    montage = to_multichannels(outputs[x])
    im = Image.fromarray(np.uint8(montage*255))
    im.save('depth/'+str(fincount)+'.jpeg')
    fincount +=1


In [0]:
outputs = predict(model, inputs,batch_size=10)

#matplotlib problem on ubuntu terminal fix
#matplotlib.use('TkAgg')   

# Display results
viz = display_images(outputs.copy(), inputs.copy())
plt.figure(figsize=(10,5))
plt.imshow(viz)
plt.savefig('test.png')
plt.show()


In [0]:
to_multichannels(outputs[0])

In [0]:
montage = to_multichannels(outputs[0])
im = Image.fromarray(np.uint8(montage*255))
im.save('testdata.jpg')
